# Obtain NetCDF file from ERA5-Land using the CDSAPI
Information here: https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation

In [ ]:
import xarray as xr
import hvplot.xarray
import pandas as pd
import fsspec
import cdsapi

In [ ]:
c = cdsapi.Client()

#### Specify variables, spatial and temporal extents

In [ ]:
var_list = ['snow_depth_water_equivalent', 'soil_temperature_level_1']

In [ ]:
# CONUS
north = 49.3457868 
west = -124.7844079 
east = -66.9513812 
south =  24.7433195 

In [ ]:
fs = fsspec.filesystem('s3', anon=False,  skip_instance_cache=True)

In [ ]:
dates = pd.date_range('1980-12-01','2022-01-31', freq='14D')
print(dates)

In [ ]:
start_dates = [date.strftime('%Y-%m-%d') for date in dates[:-1]]

In [ ]:
s3_files = [f'esip-qhub/usgs/era5_land1/conus_{start_date}.nc' for start_date in start_dates]

In [ ]:
s3file = s3_files[0]
s3file

In [ ]:
import datetime as dt   
start_date = s3file.split('_')[-1].split('.')[0]
s = start_date.split('-')
stop_date = (dt.datetime(int(s[0]),int(s[1]),int(s[2])) + pd.offsets.Day(13)).strftime('%Y-%m-%d')
local_ncfile = f'era5land_{start_date}.nc'
c.retrieve(
    'reanalysis-era5-land',
    {
        'variable': var_list, 
        'area'    : f'{north}/{west}/{south}/{east}', 
        'date'    : f'{start_date}/{stop_date}',
        'time': ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00',
                 '06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
                 '12:00', '13:00', '14:00', '15:00', '16:00', '17:00',
                 '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
        'format':'netcdf'
    },
    local_ncfile)

ds = xr.open_dataset(local_ncfile)

In [ ]:
ds

In [ ]:
ds.sd

In [ ]:
da = ds['sd'].isel(time=-1)
da.hvplot.image(x='longitude', y='latitude', geo=True, rasterize=True, tiles='ESRI')

In [ ]:
ds.sd.encoding